# Project Title

## Import Packages

In [ ]:
import pandas as pd
import numpy as np
import re
import requests
import plotly
import plotly.express as px
#import nbformat

## Get and Read in CSV

In [ ]:
# Replace 'csv_url' with the direct URL of the CSV file
csv_url = "https://www.ons.gov.uk/file?uri=/economy/inflationandpriceindices/datasets/consumerpriceindices/current/mm23.csv"

# Make a GET request to the CSV file URL
response = requests.get(csv_url)

# Check if the request was successful
if response.status_code == 200:
    # Save the CSV content to a local file
    with open('../data/raw/mm23.csv', 'wb') as csv_file:
        csv_file.write(response.content)
    print("CSV file downloaded successfully.")
else:
    print("Failed to fetch the CSV file.")

In [ ]:
df_CPI = pd.read_csv(filepath_or_buffer = "../data/raw/mm23.csv")

## Data Cleaning

In [ ]:
#getting the right months, issue here is that cells might change better to do it with matching? 
df_months = df_CPI[1017:1444].copy()
df_months['Title'] = pd.to_datetime(df_months['Title'], format='%Y %b')

In [ ]:
CPI_division_level = [
    "CPI INDEX 01 : FOOD AND NON-ALCOHOLIC BEVERAGES 2015=100",
    "CPI INDEX 02:ALCOHOLIC BEVERAGES,TOBACCO & NARCOTICS 2015=100",
    'CPI INDEX 03 : CLOTHING AND FOOTWEAR 2015=100',
    "CPI INDEX 04 : HOUSING, WATER AND FUELS 2015=100",
    "CPI INDEX 05 : FURN, HH EQUIP & ROUTINE REPAIR OF HOUSE 2015=100",
    "CPI INDEX 06 : HEALTH 2015=100",
    "CPI INDEX 07 : TRANSPORT 2015=100",
    "CPI INDEX 08 : COMMUNICATION 2015=100",
    "CPI INDEX 09 : RECREATION & CULTURE 2015=100",
    "CPI INDEX 10 : EDUCATION 2015=100",
    "CPI INDEX 11 : HOTELS, CAFES AND RESTAURANTS 2015=100",
    "CPI INDEX 12 : MISCELLANEOUS GOODS AND SERVICES 2015=100"
]
     
CPIH_division_level = ["CPIH INDEX 01 : FOOD AND NON-ALCOHOLIC BEVERAGES 2015=100",
                       "CPIH INDEX 02:ALCOHOLIC BEVERAGES,TOBACCO & NARCOTICS 2015=100",
                       "CPIH INDEX 03 : CLOTHING AND FOOTWEAR 2015=100",
                       "CPIH INDEX 04: Housing, water, electricity, gas and other fuels 2015=100", #this is different to above
                       "CPIH INDEX 05 : FURN, HH EQUIP & ROUTINE REPAIR OF HOUSE 2015=100",
                       "CPIH INDEX 06 : HEALTH 2015=100",
                       "CPIH INDEX 07 : TRANSPORT2015=100",
                       "CPIH INDEX 08 : COMMUNICATION 2015=100",
                       "CPIH INDEX 09 : RECREATION & CULTURE 2015=100",
                       "CPIH INDEX 10 : EDUCATION 2015=100",
                       "CPIH INDEX 11 : HOTELS, CAFES AND RESTAURANTS 2015=100",
                       "CPIH INDEX 12 : MISCELLANEOUS GOODS AND SERVICES 2015=100"
                       ]

CPH_CPIH_list = [CPI_division_level, CPIH_division_level]

CPH_CPIH_list_df = []

for division_type in CPH_CPIH_list:

    # Create a list of DataFrames to merge
    dfs_to_merge = [df_months[['Title', x]] for x in division_type]

    # Merge the DataFrames on the "Title" column
    merged_df = dfs_to_merge[0]
    for df in dfs_to_merge[1:]:
        merged_df = pd.merge(merged_df, df, on="Title", how='inner')
        



    #renaming columns - could create a function for this
    if division_type == CPI_division_level:
        merged_df = merged_df.rename(columns = {"CPI INDEX 02:ALCOHOLIC BEVERAGES,TOBACCO & NARCOTICS 2015=100":"CPI INDEX 02 : ALCOHOLIC BEVERAGES,TOBACCO & NARCOTICS 2015=100",
                                            "Title":"Date"})
        clean_column_names = []
        pattern = r'CPI INDEX \d+ : (.*?) \d+=\d+'
        clean_column_names.extend([re.sub(pattern, r'\1', text) for text in merged_df.columns])
        
        merged_df.columns = clean_column_names
    else:
        merged_df = merged_df.rename(columns = {"CPIH INDEX 02:ALCOHOLIC BEVERAGES,TOBACCO & NARCOTICS 2015=100": "CPIH INDEX 02 : ALCOHOLIC BEVERAGES,TOBACCO & NARCOTICS 2015=100",
                                            "Title":"Date",
                                            "CPIH INDEX 07 : TRANSPORT2015=100" : "CPIH INDEX 07 : TRANSPORT 2015=100",
                                             "CPIH INDEX 04: Housing, water, electricity, gas and other fuels 2015=100":  "CPIH INDEX 04 : Housing, water, electricity, gas and other fuels 2015=100"})
        clean_column_names = []
        pattern = r'CPIH INDEX \d+ : (.+?) 2015=100'
        clean_column_names.extend([re.sub(pattern, r'\1', text) for text in merged_df.columns])
        
        merged_df.columns = clean_column_names
        

    
    CPH_CPIH_list_df.append(merged_df)
    



In [ ]:
long_df_list = []

for x in CPH_CPIH_list_df:
    merged_df_long = x.copy()
    merged_df_long = merged_df_long.set_index('Date').melt(ignore_index=False).reset_index()
    long_df_list.append(merged_df_long)

### Combining DataFrames

In [ ]:
long_CPI = long_df_list[1].copy()
long_CPI.columns = ['Date', 'Index Name', 'Value']
long_CPI['Name'] = "CPI"
long_CPI

In [ ]:
#dataframe which has CPIH and CPI columns in long format
long_CPIH = long_df_list[0].copy()
long_CPIH.columns = ['Date', 'Index Name', 'Value']
long_CPIH['Name'] = 'CPIH'
long_CPIH

In [ ]:
combined_df = pd.concat([long_CPIH, long_CPI], ignore_index=True)
combined_df = combined_df.sort_values(by=['Date'])

In [ ]:
combined_df.head(20)

## Saving CSV

In [ ]:
#CPH_CPIH_list_df[0].to_csv("../data/processed/CPI_df.csv")
#long_df_list[0].to_csv("../data/processed/CPI_long_df.csv")
#long_CPI.to_csv("../data/processed/CPI_long_df.csv")

#CPH_CPIH_list_df[1].to_csv("../data/processed/CPIH_df.csv")
#long_df_list[1].to_csv("../data/processed/CPIH_df_long.csv")
#long_CPIH.to_csv("../data/processed/CPIH_df_long.csv")



combined_df.to_csv("../data/processed/CPIH_and_CPI_df.csv")
